In [21]:
import xarray as xr
import pandas as pd
import numpy as np

In [22]:
years = np.array([2020, 2021])        # 2 years
months = np.array([1, 2, 3])          # 3 months
days = np.array([1, 2, 3, 4])         # 4 days

# Essentially simulate that there exists unique permnos for 
# non-unique permcos since each permco (company) can have 
# multiple permnos (stocks).
permnos       = [1, 2, 3, 4]
permco_values = [1, 1, 2, 2]

Y = years.shape[0]
M = months.shape[0]
D = days.shape[0]
A = len(permnos)

me_values = np.random.randint(low=0, high=(Y * M * D * A), size=(Y, M, D, A))

In [23]:
me_values.shape

(2, 3, 4, 4)

In [24]:
ds = xr.Dataset(
    data_vars={
        "me": (("year", "month", "day", "permno"), me_values)
    },
    coords={
        "year": years,
        "month": months,
        "day": days,
        "permno": permnos,
        "permco": ("permno", permco_values)
    },
    attrs={"my_custom_dataset_attr": "structure is original."}
)

In [28]:
summe = ds.me.groupby(ds.permco).sum(dim="permno")  # This gives a DataArray with dims (year, month, day, permco)
maxme = ds.me.groupby(ds.permco).max(dim="permno")  # This gives a DataArray with dims (year, month, day, permco)

maxme_broadcast = maxme.sel(permco=ds.permco)  # This will align maxme with the original permno dimension
is_max_permno = (ds.me == maxme_broadcast) 
summe_broadcast = summe.sel(permco=ds.permco)  # This aligns summe with the permno dimension

new_me = xr.where(is_max_permno, summe_broadcast, ds.me)

ds_updated = ds.copy()
ds_updated["me"] = new_me

FrozenMappingWarningOnValuesAccess({'year': 2, 'month': 3, 'day': 4, 'permno': 4})


In [32]:
ds

<xarray.Dataset> Size: 904B
Dimensions:  (year: 2, month: 3, day: 4, permno: 4)
Coordinates:
  * year     (year) int64 16B 2020 2021
  * month    (month) int64 24B 1 2 3
  * day      (day) int64 32B 1 2 3 4
  * permno   (permno) int64 32B 1 2 3 4
    permco   (permno) int64 32B 1 1 2 2
Data variables:
    me       (year, month, day, permno) int64 768B 33 46 6 84 37 ... 86 13 40 87
Attributes:
    my_custom_dataset_attr:  structure is original.

In [1]:
from src import DataManager
from src.data.core.operations import mean, median, mode, ema
import matplotlib.pyplot as plt
import time

In [2]:
# Initialize the DataManager to handle dataset operations
dm = DataManager()
    
    # Pull in the CRSP data for Apple and Tesla.
    # Data parameters: symbols, date range, and data provider configuration.
datasets = dm.get_data([{"data_path": "wrds/equity/crsp",
        "config": {
            "start_date": "2000-01-01",
            "end_date": "2024-01-01",
            "freq": "M",
            "filters": {
                "date__gte": "2000-01-01"
            },
            "processors": {
                "replace_values": {
                    "source": "delistings",
                    "rename": [["dlstdt", "time"]],
                    "identifier": "permno",
                    "from_var": "dlret",
                    "to_var": "ret"
                },
                "merge_table": [
                    {
                        "source": "msenames",
                        "identifier": "permno",
                        "column": "comnam",
                        "axis": "asset"
                    },
                    {
                        "source": "msenames",
                        "identifier": "permno",
                        "column": "exchcd",
                        "axis": "asset"
                    }
                ],
                "set_permco_coord":  True,
                "fix_market_equity": True
            }
        }}])

wrds/equity/crsp: Level 1 cache not found. Loading data using loader (id: 131820050995712).


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


wrds/equity/crsp: Successfully saved cache to /home/suchismit/data/cache/wrds/equity/crsp/aa7b81688d3fe9cbe1f6643285838853_1925-01-01_2023-12-01.nc


/home/ubuntu/conda/envs/jax/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/ubuntu/conda/envs/jax/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/ubuntu/conda/envs/jax/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/ubuntu/conda/envs/jax/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  s

RuntimeError: wrds/equity/crsp: Post-processing steps failed for dataset. Error: 'str' object has no attribute 'columns'

In [4]:
crsp = datasets["wrds/equity/crsp"]
crsp["adj_prc"] = crsp["prc"] / crsp["cfacpr"]

In [5]:
crsp

<xarray.Dataset> Size: 5GB
Dimensions:       (year: 99, month: 12, day: 1, asset: 37747, time: 1188)
Coordinates:
  * year          (year) int32 396B 1925 1926 1927 1928 ... 2020 2021 2022 2023
  * month         (month) int32 48B 1 2 3 4 5 6 7 8 9 10 11 12
  * day           (day) int64 8B 1
  * asset         (asset) float64 302kB 1e+04 1e+04 ... 9.344e+04 9.344e+04
    time          (year, month, day) datetime64[ns] 10kB 1925-01-01 ... 2023-...
    mask          (time) bool 1kB True False False True ... True False True True
    mask_indices  (time) int64 10kB 0 3 4 5 6 8 9 11 ... -1 -1 -1 -1 -1 -1 -1 -1
Data variables: (12/20)
    cusip         (year, month, day, asset) <U8 1GB '' '' '' ... '' '88160R10'
    permco        (year, month, day, asset) float32 179MB nan nan ... 5.345e+04
    issuno        (year, month, day, asset) float32 179MB nan nan ... 6.625e+04
    hexcd         (year, month, day, asset) float32 179MB nan nan ... nan 3.0
    hsiccd        (year, month, day, asset) float32 179MB nan nan ... 9.999e+03
    bidlo         (year, month, day, asset) float32 179MB nan nan ... nan 235.6
    ...            ...
    cfacshr       (year, month, day, asset) float32 179MB nan nan ... nan 1.0
    altprc        (year, month, day, asset) float32 179MB nan nan ... nan 248.5
    spread        (year, month, day, asset) float32 179MB nan nan ... nan nan
    altprcdt      (year, month, day, asset) float32 179MB nan nan ... 2.337e+04
    retx          (year, month, day, asset) float32 179MB nan nan ... 0.03499
    adj_prc       (year, month, day, asset) float32 179MB nan nan ... nan 248.5